# Image detection exmple: Excavator
This notebook shows how you can import data from a `Excel` sheet, run our detector.py file on the data, and lastly add it to a new Excel file.
\\
We start by importing our nessecary packages.

In [ ]:
import cv2              #used to read and write images
import numpy as np      #Used to manipulate image tensors into images
import pandas as pd     #Used to get a dataframe from the excel sheets
import requests         #Used to load images

We start by loading our Excel sheet into a `Pandas` dataframe. This sheet contains info on different ships and their unique `IMO` number.

In [ ]:
df = pd.read_excel("excel/skipsListe.xlsx")
imo_column = df["imo"]
imo_list = list(imo_column)

We create a helper function for loading images from `ship-info.com`. This is simply done using the IMO number.

In [ ]:
def load_image_by_imo(imo):
    image_url = "https://www.ship-info.com/Vessels/" + str(imo) + ".jpg"    # URL to the image of the ship
    response = requests.get(image_url)                                      
    if response.status_code != 200: return None                             # If we dont find a image we can't classify
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)   # We convert the raw data into a image
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    return image

Next we load the `detector.py` script. This script is made to simplify the `YOLO` model in the `ultralytics` packages.
\\
The `find_objects_image()` function returns a set of in the form of `(object, probability)`. This gives all the detected objects in the image and their corresponding probability. More info about this method can be found in the `README.md` file.
\\
In this case we loop trough the 100 first values in the IMO list (this can be changed), and save the classified values.

In [ ]:
from detector import Detector
detector = Detector("models/excavatorV8s.pt")
classifier_dictonary = {}

for imo in imo_list[0:100]:
    image = load_image_by_imo(imo)
    if image is not None:   # if we can load the image, save the classified tuples
        classifier_dictonary[imo] = detector.find_objects_image(image)    #
    else:                   # If we cant find image, return defualt
        classifier_dictonary[imo] = [("No image found",0)]                                

We take our dictonary of IMO numbers and their classified objects, and add them to the dataframe. Lastly we convert the dataframe to a new Excel sheet.

In [ ]:
df["classes"] = ["No image found"]*(df.shape[0])    #add default values
df["confidence"] = [0]*(df.shape[0])

for imo, array in classifier_dictonary.items():     #run trough all images and there predicted objects
    if len(array) == 0: continue                    #nothing classified
    class_string = ""                               #return strings to be put in excel
    confidence_string = ""
    for tuple in array:
        class_string += tuple[0] + " "              #add found objects
        confidence_string += str(round((tuple[1]),2)) + " "
   
    df.loc[df["imo"] == imo, "classes"] = class_string #add it to their respective imo number
    df.loc[df["imo"] == imo, "confidence"] = confidence_string
    
df.to_excel("excel/test.xlsx", index=False)         #Make a new Excel file from the dataframe

Note: This creates a Excel file with no styling. The reader may add styling if neccesary.